Run this cell to initialize population synthesis

In [19]:
import sys
import os
from types import SimpleNamespace
sys.path.append(os.path.abspath(".."))

from synthesize import *
from preprocess import *

test_on = "lyon"

if test_on == "lyon":
    config_folder = "../configs/Lyon"
    ploc = None
elif test_on == "chicago":
    config_folder = "../configs/Chicago"
    ploc = puma_locations(config_folder)

model_conf, synth_conf, survey_conf = load_config(config_folder=config_folder)

# namespace for synth configuration
synth = SimpleNamespace(**synth_conf)
n = synth.sample_size
subsample = synth.subsample
batch_size = synth.batch_size
source = synth.source
shuffle_response = synth.shuffle_response
shuffle_prompt = synth.shuffle_response
wrap = synth.wrap
read_from_dataset = synth.read_from_dataset

# person = process_pums_data(config_folder)
if test_on == "lyon":
    person = process_insee_census(config_folder=config_folder)
else:
    person = process_pums_data(config_folder = config_folder)

population_sample = synthesize_population(
    config_folder=config_folder, n_sample=5000, min_age=18, max_age=65, random_state=2,
    source=source,
    read_from_dataset=read_from_dataset
)

year = 2019

output = []
attribute_descriptions = get_attribute_descriptions(person)

In [20]:
from collections import Counter

smg = SystemMessageGenerator(
    config_folder,
    "SystemMessage.j2",
    verbose_debug=False,
    shuffle=False,
    wrap=80
    )

msg = []
for i, individual in population_sample.reset_index().iterrows():
    individual_attributes = attribute_decoder_dict(individual.to_dict(), person)
    system_message = smg.write_system_message(**individual_attributes, **attribute_descriptions, ploc=ploc, YEAR=year)
    print(system_message+"\n")

Tu es un homme de 50 ans né(e) en 1969 dans le Rhône. Tu habites dans l'Ain dans
une maison. Tu vis seul. Tu travailles à plein temps comme salarié dans les
autres activités de services.

Tu es une femme de 54 ans né(e) en 1965 dans le Savoie. Tu habites dans l'Ain
dans une maison. Tu vis avec ton mari et tes 1 enfant . Tu travailles à plein
temps comme salarié dans l'administration publique, enseignement, santé humaine
et action sociale.

Tu es un homme de 52 ans né(e) en 1967 dans le Ain. Tu habites dans l'Ain dans
une maison. Tu vis avec ton épouse et tes 1 enfant . Tu travailles à plein temps
comme salarié dans l'industries extractives, énergie, eau, gestion des déchets
et dépollution.

Tu es un homme de 56 ans né(e) en 1963 dans le Landes. Tu habites dans l'Ain
dans une maison. Tu vis avec ton épouse. Tu travailles à plein temps comme
salarié dans les transports et entreposage.

Tu es une femme de 56 ans né(e) en 1963 dans le Côte-d'Or. Tu habites dans l'Ain
dans une maison. Tu vi

In [45]:
var = "MOCO"

counts = population_sample[var].value_counts()
print(counts)

responses = person.get(var)
responses

MOCO
22    1071
21     590
32     516
11     208
23     151
12      92
31      88
40      41
Name: count, dtype: int64


{'description': 'Mode de cohabitation',
 'dtype': 'CHAR',
 'answers': {'11': "Enfants d'un couple",
  '12': "Enfants d'une famille monoparentale",
  '21': "Adultes d'un couple sans enfant",
  '22': "Adultes d'un couple avec enfant(s)",
  '23': "Adultes d'une famille monoparentale",
  '31': 'Hors famille dans ménage de plusieurs personnes',
  '32': 'Personnes vivant seules',
  '40': 'Personnes vivant hors ménage'}}

In [32]:
var = "STATR"

counts = population_sample[var].value_counts()
print(counts)

responses = person.get(var)
responses

STATR
1    1810
Z     743
2     204
Name: count, dtype: int64


{'description': 'Statut professionnel regroupé',
 'dtype': 'CHAR',
 'answers': {'1': 'Salariés', '2': 'Non salariés', 'Z': 'Sans objet'}}

In [31]:
population_sample.groupby("TACT")["STATR"].value_counts()

TACT  STATR
11    1        1810
      2         204
12    Z         225
21    Z         126
22    Z         155
24    Z          99
25    Z         138
Name: count, dtype: int64